In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time

C:\Users\PAUL\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
cc_data = pd.read_csv('C:/Users/PAUL/creditcardfraud/creditcard.csv')
#print (cc_data)

In [4]:
shuffled_data = cc_data.sample(frac = 1)
#shuffled_data


In [5]:
one_hot_data = pd.get_dummies(shuffled_data, columns = ['Class'])
# one_hot_data

In [6]:
normalized_data = (one_hot_data - one_hot_data.min())/(one_hot_data.max()-one_hot_data.min())
# normalized_data

In [7]:
df_X = normalized_data.drop(['Class_0','Class_1'], axis =1)
# df_X

In [8]:
df_y = normalized_data[['Class_0', 'Class_1']]
# df_y

In [9]:
ar_X, ar_y = np.asarray(df_X, dtype = 'float32'), np.asarray(df_y, dtype = 'float32')
# ar_X


In [10]:
# ar_y

In [11]:
train_size = int(0.8*len(ar_X))


In [12]:
(raw_X_train, raw_y_train) = (ar_X[:train_size], ar_y[:train_size])
(raw_X_test, raw_y_test) = (ar_X[train_size:], ar_y[train_size:])

In [13]:
count_legit, count_fraud = np.unique(cc_data['Class'], return_counts=True)[1]
fraud_ratio = float(count_fraud/(count_legit+count_fraud))
print('Percent of fraudlulent transactions:',fraud_ratio)

Percent of fraudlulent transactions: 0.001727485630620034


In [14]:
weighting = 1/fraud_ratio
raw_y_train[:,1] = raw_y_train[:,1]*weighting

In [15]:
input_dim = ar_X.shape[1]
output_dim = ar_y.shape[1]
num_layer1 = 100
num_layer2 = 150

X_train_node = tf.placeholder(tf.float32, [None, input_dim], name= 'X_train')
y_train_node = tf.placeholder(tf.float32, [None, output_dim], name= 'y_train')

X_test_node = tf.constant(raw_X_test, name = 'X_test')
y_test_node = tf.constant(raw_y_test, name = 'y_test')


weight_1_node = tf.Variable(tf.zeros([input_dim, num_layer1]), name = 'weight_1')
biases_1_node = tf.Variable(tf.zeros([num_layer1]), name = 'bias_1')


weight_2_node = tf.Variable(tf.zeros([num_layer1, num_layer2]), name = 'weight_2')
biases_2_node = tf.Variable(tf.zeros([num_layer2]), name = 'bias_2')

weight_3_node = tf.Variable(tf.zeros([num_layer2, output_dim]), name = 'weight_3')
biases_3_node = tf.Variable(tf.zeros([output_dim]), name = 'bias_3')


In [16]:
def net(input):
    layer1 = tf.nn.sigmoid(tf.matmul(input, weight_1_node)+biases_1_node)
    layer2 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(layer1, weight_2_node)+biases_2_node), 0.85)
    layer3 = tf.nn.softmax(tf.matmul(layer2,weight_3_node)+biases_3_node)
    return layer3
y_train_prediction = net(X_train_node)
y_test_prediction = net(X_test_node)

cross_entropy = tf.losses.softmax_cross_entropy(y_train_node, y_train_prediction)

optimizer = tf.train.AdamOptimizer(0.005).minimize(cross_entropy)

def calc_accu(actual, predicted):
    actual = np.argmax(actual, 1)
    predicted = np.argmax(predicted, 1 )
    return (100 * np.sum(np.equal(predicted, actual))/predicted.shape[1])


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [21]:
num_epochs = 100

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(num_epochs):
        start_time = time.time()
        _, cross_entropy_score = session.run([optimizer, cross_entropy], feed_dict = {X_train_node: raw_X_train, y_train_node: raw_y_train})
        
        if epoch % 10 == 0:
            timer = time.time()-start_time
            
            print ('Epoch: {}'. format(epoch), 'Current loss {0:.4f}'.format(cross_entropy_score), 'Elapsed time: {0:.2f} seconds'. format(timer))
            
    final_y_test = y_test_node.eval()
    final_y_test_pred = y_test_prediction.eval()
    final_accuracy = calc_accu(final_y_test, final_y_test_pred)
    print ('Final Accuracy: {0:.2f}%'.format(final_accuracy))


Epoch: 0 Current loss 1.3948 Elapsed time: 4.24 seconds
Epoch: 10 Current loss 1.3925 Elapsed time: 3.73 seconds
Epoch: 20 Current loss 1.3669 Elapsed time: 3.84 seconds
Epoch: 30 Current loss 1.2688 Elapsed time: 3.73 seconds
Epoch: 40 Current loss 1.1022 Elapsed time: 3.80 seconds
Epoch: 50 Current loss 0.9720 Elapsed time: 3.73 seconds
Epoch: 60 Current loss 0.9097 Elapsed time: 3.77 seconds
Epoch: 70 Current loss 0.8698 Elapsed time: 3.80 seconds
Epoch: 80 Current loss 0.8530 Elapsed time: 3.75 seconds
Epoch: 90 Current loss 0.8426 Elapsed time: 3.74 seconds


IndexError: tuple index out of range

In [ ]:
final_fraud_y_test = final_y_test[final_y_test[:,1]==1]
final_fraud_y_test_pred = final_fraud_y_test_pred[final_y_test[:,1] == 1]
final_fraud_accuracy = calc_accu(final_fraud_y_test, final_y_test_pred)
print ('Final fraud specific Accuracy: {0:.2f}%'.format(final_fraud_accuracy))
